<a href="https://colab.research.google.com/github/sanjeet123456789/python-deep-learning/blob/master/Tensorflow_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 2.x
from __future__ import absolute_import,division,print_function,unicode_literals
import tensorflow as tf
import pandas as pd

train_path=tf.keras.utils.get_file("iris_training.csv","https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path=tf.keras.utils.get_file("iris_test.csv","https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")
print(train_path.count)


CSV_COLUMN_NAMES=['SepalLength','SepalWidth','PetalLength','PetalWidth','Species']
SPECIES=['Setosa','Versicolor','Virginica']

train=pd.read_csv(train_path,names=CSV_COLUMN_NAMES,header=0)
test=pd.read_csv(test_path,names=CSV_COLUMN_NAMES,header=0)
train.head()

TensorFlow 2.x selected.
8192/573 [============================================================================================================================================================================================================================================================================================================================================================================================================================================] - 0s 0us/step
<built-in method count of str object at 0x7f66a8ecdfa8>


,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [0]:
train_y=train.pop('Species')
test_y=test.pop('Species')
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


In [0]:
train_y.head()

0    2
1    1
2    2
3    0
4    0
Name: Species, dtype: int64

In [0]:
def input_fn(features,labels,training=True,batch_size=256):
  #COnverting the inputs to a dataset
  dataset=tf.data.Dataset.from_tensor_slices((dict(features),labels))
  #Shuffle and repeat if you are in training mode
  if training:
    dataset=dataset.shuffle(1000).repeat()

  return dataset.batch(batch_size)


In [0]:
#feature column describe how to use the input
my_feature_columns=[]
for key in train.keys():
  my_feature_columns.append(tf.feature_column.numeric_column(key=key))






#building a model
# tensorflow provide 2 predefine model
# 1.DNN
# 2.Classifier


#build a DNN with 2 hidden layers with 30 and 10 hidden nodes each.
classifier=tf.estimator.DNNClassifier(feature_columns=my_feature_columns,
    #two hidden layers of 30 and 10 nodes respectively
    #layers like input layer ,middle layer ,output layer 
    hidden_units=[30,10],
    n_classes=3
)

#now we trian the model
# classifier.train(
#     input_fn=lambda:input_fn(train,train_y,training=True),
#     steps=5000
# )

classifier.train(
    input_fn=lambda:input_fn(train,train_y,training=True),
    steps=5000
)


In [0]:
eval_result=classifier.evaluate(input_fn=lambda:input_fn(test,test_y,training=False))
print('\nTest set accuracy:{accuracy:0.3f}\n'.format(**eval_result))

In [0]:
def input_fn(features,batch_size=256):
  #Convert the inputs to a Dataset without labels
  return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)
features=['SepalLength','SepalWidth','PetalLength','PetalWidth']
predict={}
print("Please type numeric values as prompted")
for feature in features:
  valid=True
  while valid:
    val=input(feature + ": ")
    if not val.isdigit():valid=False

  predict[feature]=[float(val)]

prediction=classifier.predict(input_fn=lambda:input_fn(predict))
for pred_dict in prediction:
  print(pred_dict)#every result comes as a dictoneries format
  class_id=pred_dict['class_ids'][0]
  probability=pred_dict['probabilities'][class_id]

  print('prediction is "{}" ({:.1f}%)'.format(
      SPECIES[class_id],100*probability))


Please type numeric values as prompted
SepalLength: 5.9
SepalWidth: 3.0
PetalLength: 4.2
PetalWidth: 1.5
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpyi32drnr/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'logits': array([0.23279016, 1.2929839 , 1.0116547 ], dtype=float32), 'probabilities': array([0.16485524, 0.47592562, 0.35921907], dtype=float32), 'class_ids': array([1]), 'classes': array([b'1'], dtype=object), 'all_class_ids': array([0, 1, 2], dtype=int32), 'all_classes': array([b'0', b'1', b'2'], dtype=object)}
prediction is "Versicolor" (47.6%)
